In [ ]:
import pandas as pd
import json
import fiftyone.zoo as foz
import os
from pathlib import Path
import shutil

In [ ]:
num_samples = 10

In [ ]:
coco_train = foz.load_zoo_dataset("coco-2017", split="train", max_samples=num_samples)

In [ ]:
coco_test = foz.load_zoo_dataset("coco-2017", split="test", max_samples=num_samples)

In [ ]:
coco_val = foz.load_zoo_dataset("coco-2017", split="validation", max_samples=num_samples)

In [ ]:
def delete_private(d):
    if isinstance(d, list):
        for elem in d:
            delete_private(elem)
        return
    if not isinstance(d, dict):
        return
    for key in [k for k in list(d.keys()) if k.startswith('_')]:
        del d[key]
    for key in [k for k, v in list(d.items()) if isinstance(v, dict) or isinstance(v, list)]:
        delete_private(d[key])

In [ ]:
records = []
for ds in [coco_train, coco_test, coco_val]:
    for s in ds:
        d = s.to_dict(False, False)
        delete_private(d)
        records.append(d)

### Copy images and fix 'tags'

In [ ]:
img_dir = Path('./images')
try:
    os.mkdir(img_dir)
except FileExistsError:
    pass

In [ ]:
for r in records:
    path = Path(r['filepath'])
    new_path = img_dir / path.name
    shutil.copy(path, new_path)
    r['filepath'] = str(new_path)
    r['tag'] = r['tags'][0]
    del r['tags']

In [ ]:
with open('coco-records.json', 'w', encoding='utf-8') as f:
    json.dump(records, f, ensure_ascii=False, indent=4)